In [ ]:
%%shell
# SETUP OF GOOGLE COLAB ENVIRONMENT
git clone -b colab --single-branch https://github.com/lucadomene/DFB_Lab01_DataCarving /tmp/DFB_lab_init
sh /tmp/DFB_lab_init/initialize.sh

# Laboratory on Data Carving
_Digital Forensics and Biometrics_ A.A. 2025/2026

Lecturer: prof. **Simone Milani** (simone.milani@dei.unipd.it)

Teaching Assistants: **Mattia Tamiazzo** (mattia.tamiazzo.1@phd.unipd.it); **Luca Domeneghetti** (luca.domeneghetti@studenti.unipd.it)

### Prerequisites
This laboratory requires no prior knowledge concerning digital forensics. Although, basic knowledge of the Unix enviromnent may help in understanding the deletion and recovery process.

In general, the following aspects are assumed to be known and will not be covered in detail:
- Unix paradigm of files, directories and devices
- Basic Shell usage
- `Ext4` filesystem layout: inodes, block sectors, journaling
- Calculations with hexadecimal byte offsets

### Contents
The goal of this laboratory is to provide the fundamental theoretical and practical notions on data carving and file recovery. The details related to file system's structure and I/O mechanisms – although crucial for a successful data recovery – are beyond the scope of this laboratory and will be provided as an additional reading material.

By the end of the laboratory the student will have acquired the following skills:
- Perform a forensic disk copy using Unix imaging tools (`dcfldd`)
- Analyze the partition layout of a disk image
- List the files within a filesystem
- **Retrieve deallocated/deleted files**
- **Perform data carving** (manually and using dedicated tools)
___
## Theoretical aspects of filesystem forensics
Before beginning the practical part, a brief theoretical overview is essential to understand how a recovery procedure should be performed.

Understanding how a specific filesystem works helps to clarify how deletion processes work, how files are allocated logically and physically, and most importantly where to look when searching for deallocated data.

### The Ext filesystem
This laboratory focuses on Ext filesystems as it is widely implemented and tested under Linux environment. Its simplicity makes it the suitable starting point for a comprehensive data forensic approach.

#### Inodes
An **inode** (index nodes) is a data structure used by Unix filesystems (including the Ext family) to represent metadata of files. Each inode stores the following information:

- File type (e.g., regular file, directory, symlink)
- Permissions and ownership (UID, GID)
- Timestamps (created, modified, accessed)
- File size
- Link count
- Pointers to data blocks

(Source: [Linux Kernel](https://github.com/torvalds/linux/blob/211ddde0823f1442e4ad052a2f30f050145ccada/fs/ext4/ext4.h#L787C1-L846C3))

Inodes do **not** store the file name or its path — these are maintained in directory entries that map symbolic names to inode numbers. This separation is critical in forensic analysis and data carving: inodes may remain allocated even when directory structures are damaged or missing (this is the case for _orphan files_).

Within an Ext partition, inodes are located in inode tables. Each inode is marked as allocated or deallocated by using inode bitmaps.

[![](https://www.virtualcuriosities.com/wp-content/uploads/2025/03/linux-diagram-hard-links-inodes-20250326.webp)](https://www.virtualcuriosities.com/articles/4507/how-hard-links-and-inodes-work-on-linux)

#### Directories
In Ext filesystems, directories are special files (file code `0x2`) that store a list of **directory entries**, each mapping a filename to an **inode number**. These entries are stored sequentially in data blocks and are arranged as such:

| Offset | Size                | Name                   | Description                                               |
|--------|---------------------|------------------------|-----------------------------------------------------------|
| 0x0    | __le32              | inode                  | Number of the inode that this directory entry points to.  |
| 0x4    | __le16              | rec_len                | Length of this directory entry.                           |
| 0x6    | __u8                | name_len               | Length of the file name.                                  |
| 0x7    | __u8                | file_type              | File type code                                            |
| 0x8    | char\[255]          | name                   | File name.                                                |

(Source: [Linux Kernel](https://github.com/torvalds/linux/blob/211ddde0823f1442e4ad052a2f30f050145ccada/fs/ext4/ext4.h#L2392C1-L2398C3))

This structure allows multiple filenames (hard links) to point to the same inode.

In Ext2 and Ext3, directory entries are stored in a linear list, which can become inefficient as directories grow. Ext4 introduced **HTree indexing**, a hashed B-tree-like structure, to improve performance in large directories.

### NTFS and FAT

Aside from Ext filesystems, NTFS and FAT filesystems are frequently employed in Windows environments or external data storage (e.g. USBs). Although they share some similarities, understanding the differences is key for a successful file recovery.

- **Metadata Handling**:  
  Ext uses **inodes** to store file metadata separately from directory entries. In contrast, **NTFS** stores metadata in the **Master File Table (MFT)**, with each file represented as a record. **FAT**, being simpler, uses a **File Allocation Table** and directory entries that contain both metadata and file location info.

- **File Deletion Behavior**:  
  In **Ext**, when a file is deleted, its directory entry and inode may persist until overwritten, which aids data recovery. **NTFS** marks MFT entries as deleted but often retains significant metadata. **FAT** simply marks the first character of the filename as deleted and updates the FAT chain.

- **Journaling**:  
  **Ext3/4** and **NTFS** are journaling filesystems, enhancing data integrity but sometimes complicating recovery due to overwrites. **FAT** lacks journaling, making it more vulnerable to corruption but also leaving raw data more directly accessible for carving.

- **File Name and Path Storage**:  
  Ext separates names (in directory entries) from inodes, while NTFS stores file name attributes directly within MFT records. FAT embeds the file name directly in the directory entry.

___
## Exercise 01: perform a forensic disk imaging
When in possess of a digital forensic device (e.g. an hard disk, USB drive, internal SSD...), the first crucial step is to perform a copy of it as to avoid any accidental modification of the original.

This process is called **disk imaging** and abides the following principles:
1. the copy is an exact duplicate of the original device
2. the original device remains unaltered by the process

To avoid that unwanted (or intentional!) modifications are introduced during subsequent procedures, it is important to keep track of the **hash signature** of the original device. This step has to be performed during the first acquisition of the digital media.

The target device is located at `/dev/loop0` in the virtual Unix machine. It is **important** to select the root device `loop0` and not any of its partitions (e.g. `loop0p1`) in order to perform a full disk image.

```
Usage: dcfldd [OPTION]...
Enhanced version of dd for forensics and security.

  bs=BYTES            force ibs=BYTES and obs=BYTES (default=32768)
  count=BLOCKS        copy only BLOCKS input blocks
  if=FILE             read from FILE instead of stdin
  of=FILE             write to FILE instead of stdout
  seek=BLOCKS         skip BLOCKS obs-sized blocks at start of output
  skip=BLOCKS         skip BLOCKS ibs-sized blocks at start of input
  pattern=HEX         use the specified binary pattern as input
  textpattern=TEXT    use repeating TEXT as input
  errlog=FILE         send error messages to FILE as well as stderr
  hash=NAME           do hash calculation (md5, sha1, sha256, sha384 or sha512)
  hashlog=FILE        send hash output to FILE instead of stderr
  hashwindow=BYTES    perform a hash on every BYTES amount of data
  status=[on|off]          display a continual status message on stderr
  statusinterval=N         update the status message every N blocks
  sizeprobe=[if|of|BYTES]  what to use as value to percentage indicator
  vf=FILE                  verify that FILE matches the specified input
  verifylog=FILE           send verify results to FILE instead of stderr
```

In [ ]:
%%shell
# Task: perform a disk image of device /dev/loop0 and save it as whatever you prefer.
# How can I make `dcfldd` also compute the MD5 and SHA256 hash of the original disk device?

dcfldd if=/dev/loop0 of=image.dd hash=md5,sha256 hashlog=hash.txt # REMOVE

In [ ]:
%%shell
# Task: compare the MD5 and SHA256 hash digest of the original device (as provided by `dcfldd`) to the one of the disk image
md5sum image.dd # REMOVE
sha256sum image.dd # REMOVE

cat hash.txt # REMOVE

___
## Exercise 02: obtain filesystem info of the disk image
Each filesystem differs in the way data is arranged physically on the device. Even similar filesystems (e.g. Ext3 and Ext4) or the same filesystem may prefer an arrangement to another when considering the total space available on the device, the flags that were toggled during installation or the characteristics of the device itself (SSD, HDD...).

For convenience, it is better to extract the partition from the complete image using `dcfldd` with `if={your_image}.dd` (hint: look at options `bs`, `seek` and `count`).

TheSleuthKit (abbreviated as TSK) provides two commands to do so: `mmls` and `fsstat`

```
usage: mmls [-i imgtype] [-o imgoffset] image [images]

usage: fsstat [-f fstype] [-i imgtype] [-o imgoffset] image
```

In [ ]:
%%shell
# `mmls` provides information on the overall layout scheme of the disk image
# Task: list the partition table
mmls image.dd # REMOVE


In [ ]:
%%shell
# Task: extract the Linux partition
dcfldd if=image.dd of=linux_part.dd skip=2048 count=100352 bs=512 # REMOVE


In [ ]:
%%shell
# `fsstat` provides information on a single filesystem partition
# Task: obtain information on the Ext4 partition within the disk image
fsstat linux_part.dd # REMOVE

# Question: what is the size of a single inode entry? What is the size of a block?

___
## Exercise 03: recover deallocated files
There are multiple ways by which a file can be deleted. A common technique (especially employed if the underlying device is an HDD) is to just **deallocate** the data blocks associated to file's data, deallocate its inode entry and add a "deleted" timestamp to it.

Under these circumstances, it easily possible to recover the original file's data, provided that none if its data blocks have been overwritten in the meantime.

The output of `fls` command allows to obtain information on the files contained within the partition's filesystem; in particular, the asterix `*` denotes a file that spans in unallocated space, thus it has been "softly deleted".

```
usage: fls -r [-f fstype] [-i imgtype] [-o imgoffset] image [images] [inode]
```

In [ ]:
%%shell
# `fls` is the `ls` equivalent when inspecting a disk image
fls linux_part.dd

# Question: what are the numbers displayed on the left-hand side of the output?

In [ ]:
# Task: roam around the filesystem. Then, try to list all files recursively


Given an inode number, it is easy to display the associated metadata information by using the command `istat`. To retrieve the associated data blocks, multiple commands can be used, namely `icat`, `blkcat` and `blkls`.

```
usage: istat [-f fstype] [-i imgtype] [-o imgoffset] image inum

usage: icat [-f fstype] [-i imgtype] [-o imgoffset] image [images] inum[-typ[-id]]

usage: blkcat [-f fstype] [-i imgtype] [-o imgoffset] [-P pooltype] [-k password] image [images] unit_addr [num]

usage: blkls [-f fstype] [-i imgtype] [-o imgoffset] image [images] [start-stop]
```

In [ ]:
%%shell
# `istat` is used to display the metadata contained within an inode
# Task: display the metadata information regarding a regular file and a deleted one. What are the differences?
istat linux_part.dd 21 # REMOVE

```
Usage:
       xxd [options] [infile [outfile]]
    or
       xxd -r [-s [-]offset] [-c cols] [-ps] [infile [outfile]]
Options:
    -a          toggle autoskip: A single '*' replaces nul-lines. Default off.
    -b          binary digit dump (incompatible with -ps). Default hex.
    -C          capitalize variable names in C include file style (-i).
    -c cols     format <cols> octets per line. Default 16 (-i: 12, -ps: 30).
    -E          show characters in EBCDIC. Default ASCII.
    -e          little-endian dump (incompatible with -ps,-i,-r).
    -g bytes    number of octets per group in normal output. Default 2 (-e: 4).
    -h          print this summary.
    -i          output in C include file style.
    -l len      stop after <len> octets.
    -n name     set the variable name used in C include output (-i).
    -o off      add <off> to the displayed file position.
    -ps         output in postscript plain hexdump style.
    -r          reverse operation: convert (or patch) hexdump into binary.
    -r -s off   revert with <off> added to file positions found in hexdump.
    -d          show offset in decimal instead of hex.
    -s [+][-]seek  start at <seek> bytes abs. (or +: rel.) infile offset.
    -u          use upper case hex letters.
    -R when     colorize the output; <when> can be 'always', 'auto' or 'never'. Default: 'auto'.
    -v          show version: "xxd 2025-08-24 by Juergen Weigert et al.".
```

In [ ]:
%%shell
icat linux_part.dd 23 | xxd | head

In [ ]:
%%shell
# Task: display the content of another file with a different file type. What can you notice at the beginning?


In [ ]:
%%shell
icat linux_part.dd 20 > retrieved_data/cat.bmp
# Using the side menu "Files", you can navigate to /content/retrieved_data and download the recovered file

In [ ]:
%%shell
# Task: use either `icat`, `blkcat` or `blkls` to recover a file from a given inode


___
## Exercise 04: recover orphan files
When a directory entry gets corrupted or deleted, an inode might lose its only hard link in the filesystem by which a file was to be accessed. This causes the file to be inaccessible, even if the original inode metadata and data blocks are still properly allocated in the filesystem.

Provided by the Linux Kernel Archive package, the `fsck` tool provides a way to check for all inodes looking for a potential mismatch between their metadata "link count" and the actual one in the filesystem. Should it find an inode with an actual link count of 0 and a metadata link count greater than 0, such inode is considered "orphaned" and gets attached to the `lost+found` directory.

In [ ]:
%%shell
# Task: perform `fsck` on the Ext4 partition, USING THE TERMINAL
fsck.ext4 -f linux_part.dd


In [ ]:
%%shell
# Task: recover the newly attached orphan file (see exercise 03)


___
## Exercise 05: perform data carving
When the inode content is deallocated and overwritten, the only hope for a successful file recovery is to look in the data blocks (both allocated and deallocated). Each file has a unique heading (and sometime trailing) signature, which makes the recovery process a "matching game" against the signatures.
| Filetype | Header                              | Trailer                          |
|----------|--------------------------------------|----------------------------------|
| JPG      | `FF D8`                              | `FF D9`                          |
| PNG      | `89 50 4E 47 0D 0A 1A 0A`             | `49 45 4E 44 AE 42 60 82`        |
| PDF      | `25 50 44 46`                        | multiple                         |
| DOC      | `D0 CF 11 E0 A1 B1 1A E1`             | multiple                         |

Data carving is a process that can be performed on single files when embedded data is present (e.g. DOC, PDF...) or on whole disk images.

### **Task A: recover an embedded image from a DOC/PDF file**
Recover the file `my_dog.docx` using the techniques seen in Exercise 03, and try to open it using Microsoft Word or any other `.docx` opener.

When a **container file** (such as DOC/DOCX, PDF) gets corrupted, it may still be possible to retrieve the embedded files. Knowing that `my_dog.docx` contains a single ZIP archive (signature `504b0304`) that holds its data structure, perform data carving on it and extract the image.

In [ ]:
%%shell
# Task: recover the deleted file 'my_dog.docx'
icat linux_part.dd 16 > my_dog.docx

In [ ]:
%%shell
# Tip 1: use `grep` with a regular expression to isolate the JPG hexadecimal data
# Tip 2: `xxd` can be used to dump the hexadecimal data of a given file, but also do the opposite process
xxd -p my_dog.docx | tr -d '\n' | grep -o '504b0304.*' | xxd -r -p - my_dog.zip


In [ ]:
%%shell
# Task: unzip the extracted archive
unzip -d my_dog my_dog.zip

It is also possible to use dedicated tools that analyze binary data searching for embedded/hidden files. `binwalk` allows both to list and extract the embedded data from a given file. Use it to recover the image from `my_dog.docx`, and try it on other container files.

```
Analyzes data for embedded file types

Usage: binwalk [OPTIONS] [FILE_NAME]

Arguments:
  [FILE_NAME]  Path to the file to analyze

Options:
  -L, --list                   List supported signatures and extractors
  -q, --quiet                  Supress output to stdout
  -v, --verbose                During recursive extraction display *all* results
  -e, --extract                Automatically extract known file types
  -M, --matryoshka             Recursively scan extracted files
  -a, --search-all             Search for all signatures at all offsets
  -E, --entropy                Plot the entropy of the specified file
  -l, --log <LOG>              Log JSON results to a file
  -t, --threads <THREADS>      Manually specify the number of threads to use
  -x, --exclude <EXCLUDE>...   Do no scan for these signatures
  -y, --include <INCLUDE>...   Only scan for these signatures
  -C, --directory <DIRECTORY>  Extract files/folders to a custom directory [default: extractions]
  -h, --help                   Print help
  -V, --version                Print version
```

In [ ]:
%%shell
binwalk my_dog.docx


In [ ]:
%%shell
# Task: use `binwalk` to extract data on other container files
binwalk -e -C retrieved_data/binwalk my_dog.docx

### **Task B: recover files from journal**
A **journaling file system** is a file system capable of keeping track of changes in a record-commit fashion. It is a mechanism to prevent data loss in the event of a system crash or power failure.

The **journal** is treated as a Unix file, has its own inode and can be extracted from a disk image partition. Depending on the implementation, a journal may contain both data and its metadata, or just the latter (at the expense of possible data loss).

In [ ]:
%%shell
# Hint: use `fsstat` to retrieve the journal inode number
fsstat linux_part.dd # REMOVE


In [ ]:
%%shell
# Task: extract the journal and extract its data
icat linux_part.dd 8 > journal.raw # REMOVE

In [ ]:
%%shell
# Hint: sometimes, we are not looking for binary files...
binwalk journal.raw # REMOVE
strings journal.raw | grep 'password' # REMOVE
strings journal.raw | grep 'mount'

### **Task C: recover all files from a disk image**
Dealing with an entire disk image, it is *unconvenient* to `xxd | grep` for each filetype. Instead, resorting to dedicated tools is a much more efficient option, especially when the amount and diversity of files is considerable.

For this experience, `foremost` will be used to perform a complete data carving on our disk image.

**Attention**: `foremost` accepts a configuration file as one of its argument. In the Linux system provided, it is located under `/etc/foremost.conf`. Please check your specific OS layout should it be any different.

```
$ foremost [-v|-V|-h|-T|-Q|-q|-a|-w-d] [-t <type>] [-s <blocks>] [-k <size>]
        [-b <size>] [-c <file>] [-o <dir>] [-i <file]

-V  - display copyright information and exit
-t  - specify file type.  (-t jpeg,pdf ...)
-d  - turn on indirect block detection (for UNIX file-systems)
-i  - specify input file (default is stdin)
-a  - Write all headers, perform no error detection (corrupted files)
-w  - Only write the audit file, do not write any detected files to the disk
-o  - set output directory (defaults to output)
-c  - set configuration file to use (defaults to foremost.conf)
-q  - enables quick mode. Search are performed on 512 byte boundaries.
-Q  - enables quiet mode. Suppress output messages.
-v  - verbose mode. Logs all messages to screen
```

In [ ]:
%%shell
# Task: use `scalpel` to perform data carving on the disk image
foremost -d -v -a -c /etc/foremost.conf -o linux_part_recovery linux_part.dd # REMOVE


---
# Challenges
For those who want to test their skills in data carving and file system forensics, four optional challenges are provided as an additional task to perform.

Some of the disk images are provided as a `gzip` archive. To extract it (and keep the original archive file) use the following command:
``` shell
gunzip -k image.dd.gz
```

## CTF challenges
The three CTF challenges are taken from [picoCTF](https://play.picoctf.org/practice).
- `disko-3.dd.gz`
- `dds2-alpine.flag.img.gz`
- `tskapprentice.img.gz`

They require you to perform an analysis of the provided disk images and extract the flag with the format `picoCTF{...}`

## NTFS Timestamp recovery (HARD)
This challenge is taken from [CTF-Questions](https://github.com/mfput/CTF-Questions) repository.

- `file6.E01`

What is the exact original modification time by the kernel of the document `New Text Document.txt`? Format: YYYY-MM-DD HH:MM:SS.SSSSSS